In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

In [2]:
# Load EEGLAB dataset and revise dataset info
eeglab_set = '../../Experiments/data/AS/Parra.set'
raw_lab = mne.io.read_raw_eeglab(eeglab_set, preload=True)
raw_lab.rename_channels({'B1': 'Fpz', 'B2': 'Fp2', 'B3': 'AF8', 'B4': 'AF4',
'B5': 'Afz', 'B6': 'Fz', 'B7': 'F2','B8': 'F4', 'B9': 'F6', 'B10': 'F8',
'B11': 'FT8', 'B12': 'FC6', 'B13': 'FC4', 'B14': 'FC2', 'B15': 'FCz', 'B16': 'Cz', 
'B17': 'C2', 'B18': 'C4', 'B19': 'C6', 'B20': 'T8', 'B21': 'TP8', 'B22': 'CP6',
'B23': 'CP4', 'B24': 'CP2', 'B25': 'P2', 'B26': 'P4', 'B27': 'P6', 'B28': 'P8', 
'B29': 'P10', 'B30': 'PO8', 'B31': 'PO4', 'B32': 'O2'})

Reading c:\Users\yyao\OneDrive - KU Leuven\KU Leuven\Codes\Canonical Correlation Analysis\..\..\Experiments\data\AS\Parra.fdt
Reading 0 ... 101129  =      0.000 ...   197.518 secs...


C:\Users\yyao\AppData\Local\Temp\ipykernel_18476\1079061828.py:3: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_lab = mne.io.read_raw_eeglab(eeglab_set, preload=True)
C:\Users\yyao\AppData\Local\Temp\ipykernel_18476\1079061828.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_lab = mne.io.read_raw_eeglab(eeglab_set, preload=True)


<RawEEGLAB | Parra.fdt, 64 x 101130 (197.5 s), ~49.5 MB, data loaded>

In [3]:
# Data visualization
# It allows visual inspection and to manually select bad channels or data spans
# Skip for now
# raw_lab.plot(n_channels=64, scalings='auto')
# raw_lab.compute_psd().plot(average=True)

In [4]:
# Re-reference
raw_lab.set_eeg_reference(ref_channels=['Cz']) # Select the reference channel to be Cz
# TODO: Is removing dc offset necessary after re-referencing?
# Highpass filter - remove slow drifts
HP_cutoff = 0.5
raw_highpass = raw_lab.copy().filter(l_freq=HP_cutoff, h_freq=None)
# raw_highpass.compute_psd().plot(average=True)
# Remove power line noise
AC_freqs = 50 # AC power line frequency
row_notch = raw_highpass.copy().notch_filter(freqs=AC_freqs)
# row_notch.compute_psd().plot(average=True)
# Resampling
fdown = 30
raw_downsampled = row_notch.copy().resample(sfreq=fdown)
# raw_downsampled.compute_psd().plot(average=True)

EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.5 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Filter length: 3381 samples (6.604 sec)

Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequ

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.0s finished


In [59]:
# Load eeg data and sample rate
eeg_channel_indices = mne.pick_types(raw_lab.info, eeg=True)
EEGdata, times = raw_lab[eeg_channel_indices]
fsEEG = raw_lab.info['sfreq']